In [ ]:
library(survival)
library(regplot)
library(rms)
library(readxl)

# 读取Excel文件
file_path <- "C:\\Users\\fsyan\\Desktop\\副本数据.xlsx" 
df <- read_excel(file_path, sheet = "Sheet1")

# 检查列名
colnames(df)

# 修改列名（如果需要）
colnames(df) <- c("time", "status", "NLR", "CA199", "Size", "New_lesions")

##变量转换为因子
df$time <- df$time / 365
df$status <- as.factor(df$status)
df$NLR <- as.factor(df$NLR)
df$CA199 <- as.factor(df$CA199)
df$Size <- as.factor(df$Size)
df$New_lesions <- as.factor(df$New_lesions)

# 使用datadist函数设置数据分布
ddist <- datadist(df)
options(datadist='ddist')

# 拟合Cox模型
fit <- coxph(Surv(time, status ==1) ~ ., data=df)

# 绘制列线图
nom1 <- regplot(fit,
                plots = c("density", "boxes"),
                clickable = FALSE,
                title = "Nomogram for Survival Prediction",
                points = TRUE,
                droplines = TRUE,
                observation = df[10, ],  # 选择第10行作为示例
				rank = "range",
				showP = FALSE,
                failtime = c(3, 2, 1, 0.5),				# 设置时间点
                prfail = FALSE)

# 手动保存列线图
# 图片适当放大拉宽，文件，另存为，PDF，取名Nomogram

# 列线图风险打分
# 使用线性预测值（lp）
nomoRisk <- predict(fit, data = df, type = "lp")

# 将线性预测值转换为风险值（risk = exp(lp)）
nomoRisk <- exp(nomoRisk)

# 将风险值添加到数据框中
df <- cbind(df, Nomogram = nomoRisk)

# 输出风险打分结果
outTab <- rbind(ID = colnames(df), df)
write.table(outTab, file = "nomoRisk.txt", sep = "\t", col.names = FALSE, quote = FALSE)
